# 0. Premilinary Works
How big is the file, number of rows, etc.

In [1]:
cd ../data

/home/angelrps/git/MasterDataScience_FinalProject/data


In [2]:
!ls -Rl
# 2017_Yellow_Taxi_Trip_Data.csv is 10532305815 bytes (9,80 GB)
# 2018_Yellow_Taxi_Trip_Data.csv is 10428263736 bytes (9,71 GB)
# 2019_Yellow_Taxi_Trip_Data.csv is 8197837930 bytes (7,63 GB)

.:
total 3660944
-rwxrwxrwx 1 angelrps angelrps 1048588401 Mar 25 13:15 2017_Yellow_Taxi_Trip_Data.csv.bz2
-rwxrwxrwx 1 angelrps angelrps 1065918551 Jun 18 12:58 2018_Yellow_Taxi_Trip_Data.csv.bz2
-rwxrwxrwx 1 angelrps angelrps  835929368 Apr  2 18:02 2019_Yellow_Taxi_Trip_Data.csv.bz2
-rw-rw-rw- 1 angelrps angelrps   13400119 Jun 23 09:34 Data_Cleaned_To_Model.csv
-rw-rw-rw- 1 angelrps angelrps   24377425 Jun 20 06:26 Data_Taxis_2017_Cleaned.csv
-rw-rw-rw- 1 angelrps angelrps   26220204 Jun 20 04:20 Data_Taxis_2018_Cleaned.csv
-rw-rw-rw- 1 angelrps angelrps   26759445 Jun 20 00:08 Data_Taxis_2019_Cleaned.csv
-rwxrwxrwx 1 angelrps angelrps     564780 Jun 18 11:15 Data_Weather_Cleaned.csv
-rwxrwxrwx 1 angelrps angelrps     436485 Jun 16 10:51 LCD_documentation.pdf
-rwxrwxrwx 1 angelrps angelrps   13843633 Jun 16 09:40 NOAA_CentralPark_Weather.csv
-rwxrwxrwx 1 angelrps angelrps    3755058 Jun 22 10:01 NY_taxi_zones.csv
-rwxrwxrwx 1 angelrps angelrps     196848 Apr  7 19:34 data_dictionar

In [4]:
# I am dealing with very heavy files so I will compress them (bz2) to work with them
# and remove the heavy .csv from my drive
#!bzip2 2017_Yellow_Taxi_Trip_Data.csv 2018_Yellow_Taxi_Trip_Data.csv 2019_Yellow_Taxi_Trip_Data.csv

In [5]:
!ls -Rl

.:
total 2897760
-rwxrwxrwx 1 angelrps angelrps 1048588401 Mar 25 13:15 2017_Yellow_Taxi_Trip_Data.csv.bz2
-rwxrwxrwx 1 angelrps angelrps 1065918551 Jun 18 12:58 2018_Yellow_Taxi_Trip_Data.csv.bz2
-rwxrwxrwx 1 angelrps angelrps  835929368 Apr  2 18:02 2019_Yellow_Taxi_Trip_Data.csv.bz2
-rwxrwxrwx 1 angelrps angelrps     564780 Jun 18 11:15 Data_Weather_Cleaned.csv
-rwxrwxrwx 1 angelrps angelrps     436485 Jun 16 10:51 LCD_documentation.pdf
-rwxrwxrwx 1 angelrps angelrps   13843633 Jun 16 09:40 NOAA_CentralPark_Weather.csv
-rwxrwxrwx 1 angelrps angelrps     196848 Apr  7 19:34 data_dictionary_trip_records_yellow.pdf
-rwxrwxrwx 1 angelrps angelrps    1489310 Apr  7 19:34 taxi_zone_map_manhattan.jpg
-rwxrwxrwx 1 angelrps angelrps     202694 Apr  4 12:46 trip_record_user_guide.pdf


In [5]:
# Count Number of Lines
!bzcat -c ./2019_Yellow_Taxi_Trip_Data.csv.bz2 | wc -l
!bzcat -c ./2018_Yellow_Taxi_Trip_Data.csv.bz2 | wc -l
!bzcat -c ./2017_Yellow_Taxi_Trip_Data.csv.bz2 | wc -l

# 2017_...113.496.875 lines
# 2018_...112.234.627 lines
# 2019_...84.399.020 lines
# The process took about 15 min to calculate the lines

84399020
112234627
113496875


#### I will start exploring the 2019 dataset
I will try to use bash to clean the data as much as I can before importing a Data Frame because I believe is faster.
From a previous exploration I concluded:
1. There are trips with 0 'trip_distance'. I will remove all trips with 'trip_distance' < 0.06 (100 meters) because I consider them either measurement errors or non representative data.
2. There are trips with either negative or very little 'fare_amount' (~0.01). I will remove rows with fare amount<1$ because I consider them measurement errors.

In [11]:
!bzcat ./2019_Yellow_Taxi_Trip_Data.csv.bz2 | head -n 5

# Conclusions:
# 1. There is a header on the first line
# 2. The separator is ','

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
1,04/29/2019 11:31:03 AM,04/29/2019 11:31:03 AM,1,0,1,N,231,264,2,2.5,2.5,0.5,0,0,0.3,5.8,2.5
2,04/29/2019 10:52:08 AM,04/29/2019 11:06:18 AM,1,1.01,1,N,186,230,1,10,0,0.5,2.66,0,0.3,15.96,2.5
2,04/29/2019 11:28:40 AM,04/29/2019 11:33:01 AM,1,0.82,1,N,238,151,1,5,0,0.5,1.66,0,0.3,9.96,2.5
1,04/29/2019 11:28:06 AM,04/29/2019 12:07:32 PM,1,12.1,1,N,138,88,1,38.5,2.5,0.5,10.4,0,0.3,52.2,2.5

bzcat: I/O or other error, bailing out.  Possible reason follows.
bzcat: Broken pipe
	Input file = ./2019_Yellow_Taxi_Trip_Data.csv.bz2, output file = (stdout)


### 1. Clean up dataset taking just a sample

In [2]:
# LOAD LIBRARIES
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from datetime import datetime as dt
pd.options.display.max_columns = None
pd.options.display.max_rows = None
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

The following cleaning actions are based on the dataset exploration made in the notebok 'Data_Analysis.ipynb'

1. Import part of the csv so the memory can handle it.
2. Let's remove trips under 0.06 (100m). I consider them either errors or not relevant
3. Remove negative 'fare_amount' or <1$. I consider them errors or not relevant.
4. Select the two columns of interest:
    - 'pickup_datetime'
    - 'PULocationID'
5. Drop unwanted years:<br>
Eventhough the dataset states the year, I have detected rows with wrong years so I´ll filter them out.
6. Groupby Hour and LocationID.
7. Break down datetime in other useful variables:
    - year
    - month
    - day
    - week
    - dayofweek
    - isweekend
    - isholiday

### Create Data Frame with Manhattan zones

In [14]:
# 1. Import Location and Borough columns form NY TAXI ZONES dataset
dfzones = pd.read_csv('./NY_taxi_zones.csv', sep=',',
                      usecols=['LocationID', 'borough'])

# 2. Filter Manhattan zones
dfManZon = dfzones[dfzones['borough']=='Manhattan']

# 3. Get unique Zone values
dfManZon = dfManZon.drop_duplicates('LocationID')

# 4. Drop Borough column to keep just LocationID values and sort them
dfManZon = dfManZon.drop(['borough'], axis=1).sort_values(by='LocationID').reset_index()

dfManZon.shape
dfManZon
# There are 67 zones in Manhattan

,index,LocationID
0,3,4
1,12,12
2,13,13
3,9,24
4,40,41
5,43,42
6,44,43
7,41,45
8,47,48
9,49,50


In [15]:
# RECAP

# 1. Prepare sample file with 10.000 rows
#    Parse 'tpep_pickup_datetime' column to datetime object type
#    Import just needed columns to lighten the process
year = 2019
dfsample = pd.read_csv('./'+str(year)+'_Yellow_Taxi_Trip_Data.csv.bz2', sep=',',
                       usecols=['tpep_pickup_datetime','PULocationID','trip_distance','fare_amount'],
                       nrows=9999,
                       parse_dates={'pickup_datetime':['tpep_pickup_datetime']})

# 2. Remove all rows with 'trip_distance' < 0.06 (100m) because I consider them measurement errors or non representative values
DistIndex_to_drop = dfsample[dfsample['trip_distance'] < 0.06].index
dfsample.drop(DistIndex_to_drop, inplace=True)
# dfsample[dfsample['trip_distance']<0.06].index # this should be none

# 3. Remove rows with 'fare_amount' negative or <1$ because I consider them measurement errors or non relevant.
FareIndex_to_drop = dfsample[dfsample['fare_amount']<1].index
dfsample.drop(FareIndex_to_drop, inplace=True)
#dfsample[dfsample['fare_amount']<1] # this should be none

# 4. Filter just Manhattan pickups
dfManZon_array = dfManZon.iloc[:,0].values
dfsample = dfsample.loc[dfsample['PULocationID'].isin(dfManZon_array)]
#dfsample['PULocationID'].unique().shape

# 5. Select columns of interest.
#    'pickup_datetime'
#    'PULocationID'
dfsample = dfsample[['pickup_datetime','PULocationID']]
#dfsample.head()

# 6. Drop unwanted years
dfsample.drop(dfsample[dfsample['pickup_datetime'] < pd.Timestamp(dt(year,1,1))].index, inplace=True)
dfsample.drop(dfsample[dfsample['pickup_datetime'] >= pd.Timestamp(dt(year+1,1,1))].index, inplace=True)

# 7. Groupby HOUR and LocationID
    # set time period index
dfsample.set_index('pickup_datetime', inplace=True) # Set timestamp as index
dfsample = dfsample.to_period("H") # convert DATE to 1 hour PERIODS
    # Sort by datetime and by Location
dfsample.sort_values(['pickup_datetime', 'PULocationID'], inplace=True)
    # Create new column to count Number of Pickups per LocationID
dfsample['NoOfPickups'] =1
    # Groupby Datetime and Location to get Pickups sum()
dfsample=dfsample.groupby(['pickup_datetime', 'PULocationID'])['NoOfPickups'].sum().reset_index()
#dfsample.head(50)

# 8. Create new variables from datetime
dfsample['year'] = dfsample['pickup_datetime'].dt.year
dfsample['month'] = dfsample['pickup_datetime'].dt.month
dfsample['day'] = dfsample['pickup_datetime'].dt.day
dfsample['hour'] = dfsample['pickup_datetime'].dt.hour
dfsample['week'] = dfsample['pickup_datetime'].dt.week
dfsample['dayofweek'] = dfsample['pickup_datetime'].dt.dayofweek
    # 8.1 create variable 'isweekend'
mask = (dfsample['dayofweek'] == 5) | (dfsample['dayofweek'] == 6)
dfsample['isweekend'] = np.where(mask, 1, 0)
#dfsample.sample(50)

    # 8.2 Create date time index calendar
drange = pd.date_range(start=str(year)+'-01-01', end=str(year)+'-12-31')
cal = calendar()
holidays = cal.holidays(start=drange.min(), end=drange.max())
    # 8.3 create new column 'IsHolidays'
dfsample['IsHoliday'] = dfsample['pickup_datetime'].isin(holidays).astype(int)

print (dfsample.tail())
print(dfsample.info())
print(holidays)

      pickup_datetime  PULocationID  NoOfPickups  year  month  day  hour  \
136  2019-04-29 13:00           229            1  2019      4   29    13   
137  2019-04-29 13:00           230            1  2019      4   29    13   
138  2019-04-29 13:00           231            1  2019      4   29    13   
139  2019-04-29 13:00           234            1  2019      4   29    13   
140  2019-04-29 13:00           236            2  2019      4   29    13   

     week  dayofweek  isweekend  IsHoliday  
136    18          0          0          0  
137    18          0          0          0  
138    18          0          0          0  
139    18          0          0          0  
140    18          0          0          0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141 entries, 0 to 140
Data columns (total 11 columns):
pickup_datetime    141 non-null period[H]
PULocationID       141 non-null int64
NoOfPickups        141 non-null int64
year               141 non-null int64
month       

In [17]:
dfsample.sort_values('PULocationID')

,pickup_datetime,PULocationID,NoOfPickups,year,month,day,hour,week,dayofweek,isweekend,IsHoliday
86,2019-04-29 12:00,12,2,2019,4,29,12,18,0,0,0
41,2019-04-29 11:00,12,2,2019,4,29,11,18,0,0,0
87,2019-04-29 12:00,13,60,2019,4,29,12,18,0,0,0
42,2019-04-29 11:00,13,57,2019,4,29,11,18,0,0,0
16,2019-04-29 10:00,13,1,2019,4,29,10,18,0,0,0
88,2019-04-29 12:00,40,2,2019,4,29,12,18,0,0,0
43,2019-04-29 11:00,40,1,2019,4,29,11,18,0,0,0
89,2019-04-29 12:00,41,16,2019,4,29,12,18,0,0,0
44,2019-04-29 11:00,41,15,2019,4,29,11,18,0,0,0
17,2019-04-29 10:00,43,4,2019,4,29,10,18,0,0,0


### 2. Test Big Data approach on a sample

In [4]:
# DEFINE FUNCTION TO CLEAN DATASET
def clean_data(dfsample):
    DistIndex_to_drop = dfsample[dfsample['trip_distance'] < 0.06].index
    dfsample.drop(DistIndex_to_drop, inplace=True)

    FareIndex_to_drop = dfsample[dfsample['fare_amount']<1].index
    dfsample.drop(FareIndex_to_drop, inplace=True)

    dfsample = dfsample[['pickup_datetime','PULocationID']].copy()

    dfsample.drop(dfsample[dfsample['pickup_datetime'] < pd.Timestamp(dt(year,1,1))].index, inplace=True)
    dfsample.drop(dfsample[dfsample['pickup_datetime'] >= pd.Timestamp(dt(year+1,1,1))].index, inplace=True)

    dfsample.set_index('pickup_datetime', inplace=True) # Set timestamp as index
    dfsample = dfsample.to_period("H") # convert DATE to 1 hour PERIODS
    dfsample.sort_values(['pickup_datetime', 'PULocationID'], inplace=True)
    dfsample['NoOfPickups'] =1
    
    dfsample=dfsample.groupby(['pickup_datetime', 'PULocationID'])['NoOfPickups'].sum().reset_index()
    
    return dfsample

# DEFINE FUNCTION TO ADD DATE FEATURES TO DATASET
def add_date_features(dfsample):
    
    dfsample['year'] = dfsample['pickup_datetime'].dt.year
    dfsample['month'] = dfsample['pickup_datetime'].dt.month
    dfsample['day'] = dfsample['pickup_datetime'].dt.day
    dfsample['hour'] = dfsample['pickup_datetime'].dt.hour
    dfsample['week'] = dfsample['pickup_datetime'].dt.week
    dfsample['dayofweek'] = dfsample['pickup_datetime'].dt.dayofweek

    mask = (dfsample['dayofweek'] == 5) | (dfsample['dayofweek'] == 6)
    dfsample['isweekend'] = np.where(mask, 1, 0)

    drange = pd.date_range(start=str(year)+'-01-01', end=str(year)+'-12-31')
    cal = calendar()
    holidays = cal.holidays(start=drange.min(), end=drange.max())
        # 6.3 create new column 'IsHolidays'
    dfsample['IsHoliday'] = dfsample['pickup_datetime'].isin(holidays).astype(int)
    return dfsample

In [12]:
# TEST FUNCTIONS
# import file
year = 2019
dfsample = pd.read_csv('./'+str(year)+'_Yellow_Taxi_Trip_Data.csv.bz2', sep=',',
                       usecols=['tpep_pickup_datetime','PULocationID','trip_distance','fare_amount'],
                       nrows=9999,
                       parse_dates={'pickup_datetime':['tpep_pickup_datetime']})

dfsample_test = clean_data(dfsample)
dfsample_test = add_date_features(dfsample_test)
dfsample_test.head()

,pickup_datetime,PULocationID,NoOfPickups,year,month,day,hour,week,dayofweek,isweekend,IsHoliday
0,2019-04-29 05:00,90,1,2019,4,29,5,18,0,0,0
1,2019-04-29 06:00,68,1,2019,4,29,6,18,0,0,0
2,2019-04-29 06:00,88,1,2019,4,29,6,18,0,0,0
3,2019-04-29 06:00,107,1,2019,4,29,6,18,0,0,0
4,2019-04-29 06:00,113,1,2019,4,29,6,18,0,0,0


In [14]:
# ITERATION CODE WITH CHUNKS

year = 2019
# 1. Get 10000 rows sample iterator with 3000 chunks
dfsi = pd.read_csv('./'+str(year)+'_Yellow_Taxi_Trip_Data.csv.bz2', sep=',',
                       usecols=['tpep_pickup_datetime','PULocationID','trip_distance','fare_amount'],
                       nrows=9999,
                       chunksize=3000,
                       parse_dates={'pickup_datetime':['tpep_pickup_datetime']})

# 2. Declare empty DataFrame to store results of each chunk
all_chunks=pd.DataFrame()

# 3. Iterate through chunks and append results
for i, dfs in enumerate(dfsi):
    print(i, len(dfs))
    
    # clean dataset
    result = clean_data(dfs)
    
    all_chunks=all_chunks.append(result)
    
# The chunks may be just part of a single day so I need to sort and groupby again
result_all = all_chunks.sort_values(['pickup_datetime', 'PULocationID']). \
                                    groupby(['pickup_datetime', 'PULocationID'])['NoOfPickups'].sum().reset_index()
# add date features to dataset
add_date_features(result_all)
result_all.sample(10)

0 3000
1 3000
2 3000
3 999


,pickup_datetime,PULocationID,NoOfPickups,year,month,day,hour,week,dayofweek,isweekend,IsHoliday
265,2019-04-29 12:00,262,79,2019,4,29,12,18,0,0,0
22,2019-04-29 10:00,48,1,2019,4,29,10,18,0,0,0
67,2019-04-29 11:00,26,1,2019,4,29,11,18,0,0,0
149,2019-04-29 11:00,236,234,2019,4,29,11,18,0,0,0
133,2019-04-29 11:00,193,1,2019,4,29,11,18,0,0,0
165,2019-04-29 12:00,7,5,2019,4,29,12,18,0,0,0
170,2019-04-29 12:00,24,11,2019,4,29,12,18,0,0,0
255,2019-04-29 12:00,237,256,2019,4,29,12,18,0,0,0
262,2019-04-29 12:00,256,1,2019,4,29,12,18,0,0,0
214,2019-04-29 12:00,140,142,2019,4,29,12,18,0,0,0


I check that the 'result_all' dataframe and 'dfsample' are the same:
- Shape is  in both (279,3)
- Head and Tail rows are the same
- Taking a random row (e.g. row 52), they are both the same.

In [18]:
print(dfsample.tail())
print(result_all.tail())

      pickup_datetime  PULocationID  NoOfPickups  year  month  day  hour  \
274  2019-04-29 13:00           229            1  2019      4   29    13   
275  2019-04-29 13:00           230            1  2019      4   29    13   
276  2019-04-29 13:00           231            1  2019      4   29    13   
277  2019-04-29 13:00           234            1  2019      4   29    13   
278  2019-04-29 13:00           236            2  2019      4   29    13   

     week  dayofweek  isweekend  IsHoliday  
274    18          0          0          0  
275    18          0          0          0  
276    18          0          0          0  
277    18          0          0          0  
278    18          0          0          0  
      pickup_datetime  PULocationID  NoOfPickups  year  month  day  hour  \
274  2019-04-29 13:00           229            1  2019      4   29    13   
275  2019-04-29 13:00           230            1  2019      4   29    13   
276  2019-04-29 13:00           231          

In [19]:
print(dfsample.head())
print(result_all.head())

    pickup_datetime  PULocationID  NoOfPickups  year  month  day  hour  week  \
0  2019-04-29 05:00            90            1  2019      4   29     5    18   
1  2019-04-29 06:00            68            1  2019      4   29     6    18   
2  2019-04-29 06:00            88            1  2019      4   29     6    18   
3  2019-04-29 06:00           107            1  2019      4   29     6    18   
4  2019-04-29 06:00           113            1  2019      4   29     6    18   

   dayofweek  isweekend  IsHoliday  
0          0          0          0  
1          0          0          0  
2          0          0          0  
3          0          0          0  
4          0          0          0  
    pickup_datetime  PULocationID  NoOfPickups  year  month  day  hour  week  \
0  2019-04-29 05:00            90            1  2019      4   29     5    18   
1  2019-04-29 06:00            68            1  2019      4   29     6    18   
2  2019-04-29 06:00            88            1  2019     

In [20]:
# check a random row, for example 52
print(dfsample.loc[52])
print(result_all.loc[52])

pickup_datetime    2019-04-29 10:00
PULocationID                    237
NoOfPickups                       4
year                           2019
month                             4
day                              29
hour                             10
week                             18
dayofweek                         0
isweekend                         0
IsHoliday                         0
Name: 52, dtype: object
pickup_datetime    2019-04-29 10:00
PULocationID                    237
NoOfPickups                       4
year                           2019
month                             4
day                              29
hour                             10
week                             18
dayofweek                         0
isweekend                         0
IsHoliday                         0
Name: 52, dtype: object


### 3. Run program with Big Data

In [5]:
# ITERATION CODE WITH CHUNKSsssssssssssssss

year = 2019
# 1. Get iterator with 3000 chunksize
dfsi = pd.read_csv('./'+str(year)+'_Yellow_Taxi_Trip_Data.csv.bz2', sep=',',
                       usecols=['tpep_pickup_datetime','PULocationID','trip_distance','fare_amount'],
                       chunksize=3000,
                       parse_dates={'pickup_datetime':['tpep_pickup_datetime']})

# 2. Declare empty DataFrame to store results of each chunk
all_chunks=pd.DataFrame()

# 3. Iterate through chunks and append results
for i, dfs in enumerate(dfsi):
    print(i, len(dfs))
    
    # clean dataset
    result = clean_data(dfs)
    
    all_chunks=all_chunks.append(result)
    
# The chunks may be just part of a single day so I need to sort and groupby again
result_all = all_chunks.sort_values(['pickup_datetime', 'PULocationID']). \
                                    groupby(['pickup_datetime', 'PULocationID'])['NoOfPickups'].sum().reset_index()
# add date features to dataset
add_date_features(result_all)

# save csv
result_all.to_csv('../data/Data_Taxis_'+str(year)+'_Cleaned.csv', index = False, header=True)

result_all.head(10)

0 3000
1 3000
2 3000
3 3000
4 3000
5 3000
6 3000
7 3000
8 3000
9 3000
10 3000
11 3000
12 3000
13 3000
14 3000
15 3000
16 3000
17 3000
18 3000
19 3000
20 3000
21 3000
22 3000
23 3000
24 3000
25 3000
26 3000
27 3000
28 3000
29 3000
30 3000
31 3000
32 3000
33 3000
34 3000
35 3000
36 3000
37 3000
38 3000
39 3000
40 3000
41 3000
42 3000
43 3000
44 3000
45 3000
46 3000
47 3000
48 3000
49 3000
50 3000
51 3000
52 3000
53 3000
54 3000
55 3000
56 3000
57 3000
58 3000
59 3000
60 3000
61 3000
62 3000
63 3000
64 3000
65 3000
66 3000
67 3000
68 3000
69 3000
70 3000
71 3000
72 3000
73 3000
74 3000
75 3000
76 3000
77 3000
78 3000
79 3000
80 3000
81 3000
82 3000
83 3000
84 3000
85 3000
86 3000
87 3000
88 3000
89 3000
90 3000
91 3000
92 3000
93 3000
94 3000
95 3000
96 3000
97 3000
98 3000
99 3000
100 3000
101 3000
102 3000
103 3000
104 3000
105 3000
106 3000
107 3000
108 3000
109 3000
110 3000
111 3000
112 3000
113 3000
114 3000
115 3000
116 3000
117 3000
118 3000
119 3000
120 3000
121 3000
122 3000
123

923 3000
924 3000
925 3000
926 3000
927 3000
928 3000
929 3000
930 3000
931 3000
932 3000
933 3000
934 3000
935 3000
936 3000
937 3000
938 3000
939 3000
940 3000
941 3000
942 3000
943 3000
944 3000
945 3000
946 3000
947 3000
948 3000
949 3000
950 3000
951 3000
952 3000
953 3000
954 3000
955 3000
956 3000
957 3000
958 3000
959 3000
960 3000
961 3000
962 3000
963 3000
964 3000
965 3000
966 3000
967 3000
968 3000
969 3000
970 3000
971 3000
972 3000
973 3000
974 3000
975 3000
976 3000
977 3000
978 3000
979 3000
980 3000
981 3000
982 3000
983 3000
984 3000
985 3000
986 3000
987 3000
988 3000
989 3000
990 3000
991 3000
992 3000
993 3000
994 3000
995 3000
996 3000
997 3000
998 3000
999 3000
1000 3000
1001 3000
1002 3000
1003 3000
1004 3000
1005 3000
1006 3000
1007 3000
1008 3000
1009 3000
1010 3000
1011 3000
1012 3000
1013 3000
1014 3000
1015 3000
1016 3000
1017 3000
1018 3000
1019 3000
1020 3000
1021 3000
1022 3000
1023 3000
1024 3000
1025 3000
1026 3000
1027 3000
1028 3000
1029 3000
1030 30

1750 3000
1751 3000
1752 3000
1753 3000
1754 3000
1755 3000
1756 3000
1757 3000
1758 3000
1759 3000
1760 3000
1761 3000
1762 3000
1763 3000
1764 3000
1765 3000
1766 3000
1767 3000
1768 3000
1769 3000
1770 3000
1771 3000
1772 3000
1773 3000
1774 3000
1775 3000
1776 3000
1777 3000
1778 3000
1779 3000
1780 3000
1781 3000
1782 3000
1783 3000
1784 3000
1785 3000
1786 3000
1787 3000
1788 3000
1789 3000
1790 3000
1791 3000
1792 3000
1793 3000
1794 3000
1795 3000
1796 3000
1797 3000
1798 3000
1799 3000
1800 3000
1801 3000
1802 3000
1803 3000
1804 3000
1805 3000
1806 3000
1807 3000
1808 3000
1809 3000
1810 3000
1811 3000
1812 3000
1813 3000
1814 3000
1815 3000
1816 3000
1817 3000
1818 3000
1819 3000
1820 3000
1821 3000
1822 3000
1823 3000
1824 3000
1825 3000
1826 3000
1827 3000
1828 3000
1829 3000
1830 3000
1831 3000
1832 3000
1833 3000
1834 3000
1835 3000
1836 3000
1837 3000
1838 3000
1839 3000
1840 3000
1841 3000
1842 3000
1843 3000
1844 3000
1845 3000
1846 3000
1847 3000
1848 3000
1849 3000


2570 3000
2571 3000
2572 3000
2573 3000
2574 3000
2575 3000
2576 3000
2577 3000
2578 3000
2579 3000
2580 3000
2581 3000
2582 3000
2583 3000
2584 3000
2585 3000
2586 3000
2587 3000
2588 3000
2589 3000
2590 3000
2591 3000
2592 3000
2593 3000
2594 3000
2595 3000
2596 3000
2597 3000
2598 3000
2599 3000
2600 3000
2601 3000
2602 3000
2603 3000
2604 3000
2605 3000
2606 3000
2607 3000
2608 3000
2609 3000
2610 3000
2611 3000
2612 3000
2613 3000
2614 3000
2615 3000
2616 3000
2617 3000
2618 3000
2619 3000
2620 3000
2621 3000
2622 3000
2623 3000
2624 3000
2625 3000
2626 3000
2627 3000
2628 3000
2629 3000
2630 3000
2631 3000
2632 3000
2633 3000
2634 3000
2635 3000
2636 3000
2637 3000
2638 3000
2639 3000
2640 3000
2641 3000
2642 3000
2643 3000
2644 3000
2645 3000
2646 3000
2647 3000
2648 3000
2649 3000
2650 3000
2651 3000
2652 3000
2653 3000
2654 3000
2655 3000
2656 3000
2657 3000
2658 3000
2659 3000
2660 3000
2661 3000
2662 3000
2663 3000
2664 3000
2665 3000
2666 3000
2667 3000
2668 3000
2669 3000


3390 3000
3391 3000
3392 3000
3393 3000
3394 3000
3395 3000
3396 3000
3397 3000
3398 3000
3399 3000
3400 3000
3401 3000
3402 3000
3403 3000
3404 3000
3405 3000
3406 3000
3407 3000
3408 3000
3409 3000
3410 3000
3411 3000
3412 3000
3413 3000
3414 3000
3415 3000
3416 3000
3417 3000
3418 3000
3419 3000
3420 3000
3421 3000
3422 3000
3423 3000
3424 3000
3425 3000
3426 3000
3427 3000
3428 3000
3429 3000
3430 3000
3431 3000
3432 3000
3433 3000
3434 3000
3435 3000
3436 3000
3437 3000
3438 3000
3439 3000
3440 3000
3441 3000
3442 3000
3443 3000
3444 3000
3445 3000
3446 3000
3447 3000
3448 3000
3449 3000
3450 3000
3451 3000
3452 3000
3453 3000
3454 3000
3455 3000
3456 3000
3457 3000
3458 3000
3459 3000
3460 3000
3461 3000
3462 3000
3463 3000
3464 3000
3465 3000
3466 3000
3467 3000
3468 3000
3469 3000
3470 3000
3471 3000
3472 3000
3473 3000
3474 3000
3475 3000
3476 3000
3477 3000
3478 3000
3479 3000
3480 3000
3481 3000
3482 3000
3483 3000
3484 3000
3485 3000
3486 3000
3487 3000
3488 3000
3489 3000


KeyboardInterrupt: 

In [24]:
# PERSONAL NOTES:
# sort by datetime after compiling unique csv
# interpolate missing values every hour